In [1]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"
max_seq_length = 2048
learning_rate = 2e-4
weight_decay = 0.01
max_steps = 60
warmup_steps = 10
batch_size = 8
gradient_accumulation_steps = 4
lr_scheduler_type = "linear"
optimizer = "adamw_8bit"
use_gradient_checkpointing = True
random_state = 3407

In [2]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
HAS_BFLOAT16 = torch.cuda.is_bf16_supported()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "auto",
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.1
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.999 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7+cu118. FA = True.
 "-____-"     Apache 2 free license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/cleaned_data.json", field="data", split="train")



Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
logging.set_verbosity_info()

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    # dataset_text_field = "content",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        num_train_epochs = 2,
        learning_rate = learning_rate,
        fp16 = not HAS_BFLOAT16,
        bf16 = HAS_BFLOAT16,
        logging_steps = 1,
        output_dir = "outputs",
        optim = optimizer,
        weight_decay = weight_decay,
        lr_scheduler_type = lr_scheduler_type,
        seed = random_state,
    ),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Map:   0%|          | 0/71354 [00:00<?, ? examples/s]

Using auto half precision backend


In [6]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.999 GB.
4.637 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

***** Running training *****
  Num examples = 71,354
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 4,460
  Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.766500
2,4.766300
3,4.613500
4,4.159900
5,3.576900
6,3.113800
7,2.705100
8,2.294000
9,1.816500
10,1.635600


Checkpoint destination directory outputs/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Saving model checkpoint to outputs/checkpoint-500
loading configuration file config.json from cache at /home/fruitrobo/.cache/huggingface/hub/models--teknium--OpenHermes-2.5-Mistral-7B/snapshots/91ed666be78da7556f3d79abbb26fff0ee26cb54/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_ve

In [33]:
output_dir = "outputs/test"

trainer.save_model(output_dir)
trainer.save_state()
torch.save(torch.cuda.random.get_rng_state_all(), f"{output_dir}/rng_state.pth")
torch.save(trainer.optimizer.state_dict(), f"{output_dir}/optimizer.pt")
torch.save(trainer.lr_scheduler.state_dict(), f"{output_dir}/scheduler.pt")



Saving model checkpoint to outputs/test
loading configuration file config.json from cache at /home/fruitrobo/.cache/huggingface/hub/models--teknium--OpenHermes-2.5-Mistral-7B/snapshots/91ed666be78da7556f3d79abbb26fff0ee26cb54/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.37.2",
  "use_cache": false,
  "vocab_size": 32002
}

tokenizer config file saved in outputs/test/tokenizer_config.json
Special tokens file s

In [4]:

from torchsummary import summary

print(model)

summary(model, (max_seq_length))



NameError: name 'model' is not defined

In [1]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
base_model_name = "teknium/OpenHermes-2.5-Mistral-7B" #path/to/your/model/or/name/on/hub"
adapter_model_name = "./outputs/test" #path/to/your/model/or/name/on/hub

device_map = { 'lm_head': 0, 'model.embed_tokens': 0, 'model.norm.weight': 0}


for i in range(32):
    if i <= 22:
        device_map[f"model.layers.{i}"] = 0
    else:
        device_map[f"model.layers.{i}"] = 'cpu'

model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map = device_map, torch_dtype=torch.float16, low_cpu_mem_usage=True) 

peft_map = {'base_model.model.lm_head': 'cpu', 'base_model.model.model.embed_tokens': 'cpu', 'base_model.model.model.norm.weight': 'cpu'}
for i in range(32):
        peft_map[f"base_model.model.model.layers.{i}"] = 'cpu'

model = PeftModel.from_pretrained(model, adapter_model_name, device_map = peft_map)




model = model.merge_and_unload()

model.save_pretrained('Openbhi-2.5-mistral-7b')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 